<a href="https://colab.research.google.com/github/lucas-azdias/Artistic-Text-Recognition/blob/main/artefatos/modelos/ViTSTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [ ]:
!pip install -q numpy==2.0.1
!pip install -q torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 819.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [ ]:
!pip install -q pytorch-lightning==2.5.0 lmdb fire hydra-core imgaug==0.4.0
!pip install -q pyclipper pyyaml rapidfuzz gdown einops timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.4/819.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 106.8 MB/s eta 0:00:00


In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import PIL
import sys
import torch
import torchvision
import tqdm
import os

print("NumPy:", np.__version__)
print("Torch:", torch.__version__)

NumPy: 2.0.2
Torch: 2.4.1+cu121


# Obtendo dataset

### Clonando dataset

In [ ]:
# URL do repositório
dataset_url = "https://github.com/lucas-azdias/Artistic-Text-Recognition-Dataset.git"
dataset_path = pathlib.Path("/content/WordArt-V1.5")

# Clonar o repositório
!rm -rf "{dataset_path}"
!git clone "{dataset_url}" "{dataset_path}"

Cloning into '/content/WordArt-V1.5'...
remote: Enumerating objects: 12087, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 12087 (delta 24), reused 11 (delta 0), pack-reused 12010 (from 1)
Receiving objects: 100% (12087/12087), 1.29 GiB | 31.72 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Updating files: 100% (12011/12011), done.


### Carregando dataset

In [ ]:
def load_data(base_path: pathlib.Path, name: str) -> tuple[list[pathlib.Path], list[str]]:
    labels_path = pathlib.Path(base_path, name, "labels.txt")

    image_paths = []
    classes = []

    with open(labels_path, "r") as f:
        for line in f:
            # Ex: "train_image\320.png Ford"
            splitted = line.strip().split(" ")
            path, label = splitted[0].replace("\\", "/"), " ".join(splitted[1:])

            # Ajustar o caminho para apontar para a pasta correta
            img_path = pathlib.Path(base_path, name, pathlib.Path(path))
            image_paths.append(img_path)
            classes.append(label)

    return image_paths, classes

train_images, train_labels = load_data(dataset_path, "train")
testA_images, testA_labels = load_data(dataset_path, "testA")

# Fine Tuning


In [ ]:
# URL do repositório
parseq_url = "https://github.com/baudm/parseq.git"
parseq_path = pathlib.Path("/content/parseq")

# Clonar o repositório
!rm -rf "{parseq_path}"
!git clone "{parseq_url}" "{parseq_path}"

if not str(parseq_path.absolute()) in sys.path:
    sys.path.append(str(parseq_path.absolute()))

Cloning into '/content/parseq'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 612 (delta 241), reused 192 (delta 192), pack-reused 300 (from 2)
Receiving objects: 100% (612/612), 1.34 MiB | 5.34 MiB/s, done.
Resolving deltas: 100% (343/343), done.


In [ ]:
!wget -q -O Miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !bash Miniconda.sh -b -f -p /usr/local > /dev/null
!bash Miniconda.sh -b -f -p miniconda > /dev/null
!rm Miniconda.sh

!sudo rm -f /usr/local/bin/conda
!ln -s /content/miniconda/bin/conda /usr/local/bin/conda

!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r


In [ ]:
!conda env remove -n parseq -y -q > /dev/null 2>&1
!conda create -n parseq python=3.10 -y -q > /dev/null 2>&1

!conda run -n parseq pip install -q pip-tools
!conda run -n parseq make torch-cu121 -C "{parseq_path}"
!conda run -n parseq pip install -q -r "{parseq_path}/requirements/core.cu121.txt" -e {parseq_path}/.[bench,train,test,tune] --use-pep517

make: Entering directory '/content/parseq'
CONSTRAINTS=/dev/null pip-compile --quiet --no-header --allow-unsafe --resolver=backtracking --strip-extras --output-file requirements/constraints.txt requirements/bench.in requirements/core.in requirements/dev.in requirements/test.in requirements/train.in requirements/tune.in --extra-index-url https://download.pytorch.org/whl/cpu
CONSTRAINTS=constraints.txt pip-compile --quiet --no-header --allow-unsafe --resolver=backtracking --strip-extras --no-annotate --output-file requirements/core.txt requirements/core.in
Generating requirements/core.cu121.txt
make: Leaving directory '/content/parseq'



In [ ]:
%set_env HYDRA_FULL_ERROR=1

env: HYDRA_FULL_ERROR=1


In [ ]:
# Função para alterar trechos dos arquivos alvos
def modify_target_files(target_files: list[pathlib.Path], old_texts: list[str], new_texts: list[str]) -> None:
    for target_file in target_files:
        text = target_file.read_text(encoding="utf-8")

        for old_text, new_text in zip(old_texts, new_texts):
            text = text.replace(
                old_text,
                new_text
            )

        target_file.write_text(text, encoding="utf-8")

In [ ]:
# Modificando trechos necessários
modify_target_files(
    [
        parseq_path / "read.py",
    ],
    [
        "import argparse",
        "nargs='+', ",
        "for fname in args.images:",
        "Image.open(fname)",
        "print(f'{fname}: {pred[0]}')",
    ],
    [
        "import argparse\nfrom pathlib import Path",
        "",
        "text = \"\"\n    imgs = Path(args.images)\n    for fname in (imgs.rglob(\"*\") if imgs.is_dir() else [imgs]):",
        "Image.open(fname.absolute())",
        "text += f\"{fname}\t{pred[0]}\t{p[0].tolist()[:-1]}\\n\"\n    print(text)\n    open(\"outputs.txt\", \"w\").write(text)",
    ],
)

# Resultados

In [ ]:
import re

normalize = lambda s: re.sub(r'[^a-z0-9]', '', s.lower())
compare = lambda a, b: normalize(a) == normalize(b)

infer_imgs = f"/content/WordArt-V1.5/testA/images"

!cd {parseq_path} && conda run -n parseq python "{parseq_path}/read.py" \
"pretrained=vitstr" \
--images="{infer_imgs}" \
--device=cuda

with open(f"{parseq_path}/outputs.txt") as file:
            raw_results = file.read().strip().split("\n")

raw_results = [line.strip().split("\t") for line in raw_results]
raw_results = [
    tuple(
        [
            pathlib.Path(path),
            pred,
            tuple([
                float(c)
                for c in confidence.replace("[", "").replace("]", "").strip().split(", ")
            ])
        ]
    )
    for path, pred, confidence in raw_results
]

correct = 0
for img_path, label in zip(testA_images, testA_labels):
    for path, pred, _ in raw_results:
        if pathlib.Path(img_path).stem == pathlib.Path(path).stem:
            correct += 1 if compare(pred, label) else 0
            break

print(f"{correct}/{len(raw_results)} ({correct / len(raw_results) * 100:.2f}%)")

Additional keyword arguments: {}
/content/WordArt-V1.5/testA/images/new3564.png	Good	[0.998206377029419, 0.9842489361763, 0.9892984628677368, 0.9902468919754028]
/content/WordArt-V1.5/testA/images/new1504.png	Keep	[0.9737006425857544, 0.9563499093055725, 0.9930786490440369, 0.977573037147522]
/content/WordArt-V1.5/testA/images/new5879.png	night	[0.9912304878234863, 0.9966775178909302, 0.9989363551139832, 0.9972152709960938, 0.9964008331298828]
/content/WordArt-V1.5/testA/images/new743.png	puRse	[0.8382062315940857, 0.7292171716690063, 0.8081976771354675, 0.8331597447395325, 0.9260048866271973]
/content/WordArt-V1.5/testA/images/new4639.png	Alteria	[0.9938527345657349, 0.6252670288085938, 0.6389975547790527, 0.5601887702941895, 0.5569556951522827, 0.5368484854698181, 0.5490140914916992]
/content/WordArt-V1.5/testA/images/new5388.png	PORSTU	[0.9877864122390747, 0.5451465845108032, 0.9858766198158264, 0.9940266609191895, 0.9908987283706665, 0.574843168258667]
/content/WordArt-V1.5/testA/i